In [1]:
import numpy as np
from skimage.transform import resize
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import tensorflow as tf

from keras.models import model_from_json


Using TensorFlow backend.


In [42]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    dcm = pydicom.dcmread(filename)       
    img = dcm.pixel_array
    if dcm.PatientPosition not in ['PA', 'AP'] or dcm.BodyPartExamined != 'CHEST' or dcm.Modality != 'DX':
            print("Unable to process this file")
            return None
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    img = img / 255
    img = resize(img, img_size)
    proc_img = (img - img_mean)/img_std
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = tf.keras.models.model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    predY = model.predict(img)
    if(predY >= thresh):
        prediction = 'Pneumonia'
    else:
        prediction = 'No Pneumonia'
    
    return prediction 

In [43]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"#path to saved model
weight_path = "xray_class_my_model.best.hdf5"#path to saved best weights

dicom = []
img = []

for test_dicom in test_dicoms:
    dicom.append(pydicom.dcmread(test_dicom))
    img.append((pydicom.dcmread(test_dicom)).pixel_array)


IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = np.mean(img)# loads the mean image value they used during training preprocessing
img_std = np.std(img)# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.538
#loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Pneumonia
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
Unable to process this file
Load file test5.dcm ...
Unable to process this file
Load file test6.dcm ...
Unable to process this file
